In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision

import torchvision.transforms as transforms

In [2]:
# transform = transforms.Compose([transforms.ToTensor(),
#                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
images = np.load("./images.npy")
labels = np.load("./y_train.npy")


In [4]:
images = images.reshape(-1,3,32,32)
images.shape

(361816, 3, 32, 32)

In [5]:
labels.shape#in label encoding

(7384, 37)

In [6]:
trainset = torch.from_numpy(images[:7000])
trainlabel = torch.from_numpy(labels[:7000])
testset = torch.from_numpy(images[7000:])
testlabel = torch.from_numpy(labels[7000:])

In [7]:
trainlabel.size()

torch.Size([7000, 37])

In [8]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [9]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5 , padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 5 ,padding=2)
        self.conv3 = nn.Conv2d(128 ,256 ,5,padding=2)
        self.conv4 = nn.Conv2d(256 ,512 ,5,padding=2)
        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 37)

    def forward(self, x):
#         x = self.conv1(x)
        x = self.pool(F.relu(self.conv1(x)))
        
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 2048)
#         print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x


net = Net()
net

Net(
  (conv1): Conv2d (3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d (64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d (128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d (256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=2048, out_features=512)
  (fc2): Linear(in_features=512, out_features=128)
  (fc3): Linear(in_features=128, out_features=37)
)

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [11]:
inp , label = Variable(trainset[0:1].view(1,3,32,32)) , Variable(trainlabel[0:1].view(1))
optimizer.zero_grad()

RuntimeError: invalid argument 2: size '[1]' is invalid for input with 37 elements at /pytorch/torch/lib/TH/THStorage.c:41

In [23]:
out = net(inp.type(torch.FloatTensor))
out.shape
# out.backward(torch.randn(1, 37))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([1, 37])

In [24]:
label.shape

torch.Size([1])

In [25]:
net.zero_grad()
loss = criterion(out, label)
loss.backward()
optimizer.step()

In [26]:
for i in range(10000):
    # get the inputs
    inputs, labels = trainset.type(torch.FloatTensor),trainlabel.type(torch.FloatTensor)

    # wrap them in Variable
    inputs, labels = Variable(inputs.view(-1,3,32,32)), Variable(labels.view(-1))
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('loss: %.3f' % (loss ))
            

print('Finished Training')

RuntimeError: $ Torch: not enough memory: you tried to allocate 10GB. Buy new RAM! at /pytorch/torch/lib/TH/THGeneral.c:246

In [ ]:
testlabel.shape

In [27]:
correct = 0
total = 0
for i in range(384):
    images, labels = testset[i].view(-1,3,32,32) , testlabel[i].view(-1)
    outputs = net(Variable(images).type(torch.FloatTensor))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 384 test images: %d %%' % (
    100 * correct / total))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 384 test images: 2 %


In [ ]:
class_correct = list(0. for i in range(37))
class_total = list(0. for i in range(37))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))